# 프로젝트 - Movielens 영화 추천 실습
이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

1) 데이터 준비와 전처리

-------------

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

# 2) 분석해 보자.

-------------

### ratings에 있는 유니크한 영화 개수

In [6]:
ratings['movie_id'].nunique()

3628

---------------

### ratings에 있는 유니크한 사용자 수

In [7]:
ratings['user_id'].nunique()

6039

In [8]:
# movies + ratings
data= pd.merge(movies, ratings)

In [9]:
#검색하기 쉽게 대문자를 소문자로 바꿔줌
data['title'] = data['title'].str.lower() 
data.head(10)

,movie_id,title,genre,user_id,counts,timestamp
0,1,toy story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,toy story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,toy story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,toy story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,toy story (1995),Animation|Children's|Comedy,10,5,978226474
5,1,toy story (1995),Animation|Children's|Comedy,18,4,978154768
6,1,toy story (1995),Animation|Children's|Comedy,19,5,978555994
7,1,toy story (1995),Animation|Children's|Comedy,21,3,978139347
8,1,toy story (1995),Animation|Children's|Comedy,23,4,978463614
9,1,toy story (1995),Animation|Children's|Comedy,26,3,978130703


------------

### 가장 인기 있는 영화 30개(인기순)

In [10]:
#인기 많은 영화
movies_count = data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

# 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [11]:
#본인이 좋아하는 아티스트 데이터로 바꿔서 추가해도 된다. 단, 이름은 꼭 데이터 셋에 있는 것과 동일하게 맞춰야한다.
my_favorite = ['toy story (1995)' , 'matilda (1996)' ,'merry poppins (1964)' ,'home alone (1990)' ,'lion king, the (1994)']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하자.
my_playlist = pd.DataFrame({'user_id': ['Han'] * 5, 'title': my_favorite, 'counts':[5]*5})

if not data.isin({'user_id': ['Han']})['user_id'].any(): #user_id에 zimin이라는 데이터가 없다면, 위에 임으로 만든 my_favorite 데이터를 추가해준다.
    data = data.append(my_playlist)
    
data.tail(10)

,movie_id,title,genre,user_id,counts,timestamp
836473,3952.0,"contender, the (2000)",Drama|Thriller,5682,3,1.029458e+09
836474,3952.0,"contender, the (2000)",Drama|Thriller,5812,4,9.920721e+08
836475,3952.0,"contender, the (2000)",Drama|Thriller,5831,3,9.862231e+08
836476,3952.0,"contender, the (2000)",Drama|Thriller,5837,4,1.011903e+09
836477,3952.0,"contender, the (2000)",Drama|Thriller,5998,4,1.001781e+09
0,NaN,toy story (1995),NaN,Han,5,NaN
1,NaN,matilda (1996),NaN,Han,5,NaN
2,NaN,merry poppins (1964),NaN,Han,5,NaN
3,NaN,home alone (1990),NaN,Han,5,NaN
4,NaN,"lion king, the (1994)",NaN,Han,5,NaN


# 4) 모델에 활용하기 위한 전처리

In [12]:
#고유한 유저, 타이틀을 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

#유저, 타이틀을 indexting 하는 코드 idx는 index의 약자이다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k, v in enumerate(title_unique)}

In [13]:
#인덱싱이 잘 되었는지 확인해 보자
print(user_to_idx['Han']) 
print(title_to_idx['lion king, the (1994)'])

6039
350


In [14]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data): #모든 row가 정상적으로 인덱싱 되었ㄷ면 
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data #data['user_id']을 인덱싱된 Series로 교체해준다
else:
    print('user_id column indexing Fail')
    
#title_to_idx를 통해 title 컬럼도 동일한 방식으로 인덱싱해준다.
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,movie_id,title,genre,user_id,counts,timestamp
0,1.0,0,Animation|Children's|Comedy,0,5,978824268.0
1,1.0,0,Animation|Children's|Comedy,1,4,978237008.0
2,1.0,0,Animation|Children's|Comedy,2,4,978233496.0
3,1.0,0,Animation|Children's|Comedy,3,5,978225952.0
4,1.0,0,Animation|Children's|Comedy,4,5,978226474.0
...,...,...,...,...,...,...
0,NaN,0,NaN,6039,5,NaN
1,NaN,769,NaN,6039,5,NaN
2,NaN,3628,NaN,6039,5,NaN
3,NaN,566,NaN,6039,5,NaN


# 5) CSR matrix를 직접 만들어 봅시다.

In [15]:
#실습
#위의 설명을 보고 이해해서 만들어보자
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_title))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 6) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [16]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

#implicit 라이브러리에서 권장하고 있는 부분이다. 학습내용과는 무관하다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [17]:
#Implcit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors = 100, regularization = 0.01, use_gpu = False, iterations = 15, dtype = np.float32)

In [18]:
#als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해준다)
csr_data_transpose = csr_data.T
csr_data_transpose

<3629x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [19]:
#모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

모델 학습이 끝났다! 이제 아래 사항 두 가지를 살펴보자
1. Han 벡터와 frozen(2013)의 벡터를 어떻게 만들고 있는지
2. 두 벡터를 곱하면 어떤 값이 나오는지


In [20]:
Han, lion = user_to_idx['Han'], title_to_idx['lion king, the (1994)']
Han_vector, lion_vector = als_model.user_factors[Han], als_model.item_factors[lion]

print('슝')

슝


In [21]:
Han_vector

array([ 0.85471725, -0.8349491 , -0.18519166, -0.24547948,  0.44605654,
        0.45773548,  0.5451082 ,  0.29692066,  0.12709183,  0.20469216,
        0.2733561 , -0.14979339,  0.79083693,  0.05037482, -0.09750586,
        0.26463145,  0.13941796,  0.36817518,  0.25998276,  0.1065319 ,
       -0.50745714, -0.1615795 , -0.7418764 , -0.40755808,  0.16717319,
       -0.0855163 , -0.46629766, -1.2296576 , -0.15105328, -0.31760606,
        0.18563628, -0.31217596,  0.19777375,  0.28166574,  0.14366502,
        0.13753682, -0.53610516, -1.0237269 ,  0.72016   , -0.12164257,
       -0.34877977, -0.1548206 , -0.9749309 , -0.00546   ,  0.18485114,
        0.4382132 ,  0.1305736 , -0.30154946, -0.56765676,  0.75749743,
       -0.34827647, -0.28570348,  0.00242105,  0.3737876 ,  0.27543816,
        0.00267909, -0.03615183,  0.4926795 ,  0.32775712, -0.50542533,
        0.16168468, -0.14219067, -0.48221442, -0.18758184, -0.8482529 ,
        0.4463797 ,  0.20467427, -0.16961583, -0.28225708, -0.01

In [22]:
lion_vector

array([ 0.0232784 , -0.00289661, -0.0134223 , -0.01005187,  0.03589487,
        0.00016752, -0.00054304, -0.00021666,  0.03431617,  0.00553277,
        0.00994898, -0.00083656,  0.03783132,  0.02556173, -0.00769492,
       -0.02374331,  0.00226518,  0.03593509,  0.01657826,  0.01797179,
       -0.00320382, -0.00315477, -0.01356339,  0.00029305,  0.00237507,
       -0.00526344, -0.01766597, -0.02145885,  0.01057454, -0.00500809,
        0.00435163,  0.00310119,  0.02723092,  0.03350946, -0.00647395,
        0.02676748, -0.02377869, -0.0277613 ,  0.00713238,  0.00756421,
        0.00437065,  0.01862679, -0.00282714,  0.00370617, -0.00915634,
        0.01258839,  0.00237465, -0.00974882, -0.03068109,  0.03568308,
        0.01474864, -0.02225677,  0.00087945,  0.01359798,  0.02446813,
       -0.01609292,  0.01292871,  0.01420778,  0.00127167, -0.01327616,
       -0.0088562 ,  0.00656224, -0.00554498,  0.00125643, -0.0079665 ,
        0.02879151,  0.00285322,  0.01878394,  0.01918367,  0.01

# 7) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [23]:
#Han_vector 과 라이언킹 을 내적하는 코드
np.dot(Han_vector, lion_vector)

0.45853296

In [24]:
bnb = title_to_idx['beauty and the beast (1991)']
bnb_vector = als_model.item_factors[bnb]
np.dot(Han_vector, bnb_vector)

0.38566348

# 8) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [26]:
favorite_title = 'lion king, the (1994)'
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N = 15)
similar_title

[(350, 0.99999994),
 (575, 0.8762136),
 (568, 0.77511835),
 (1689, 0.64536756),
 (725, 0.6430055),
 (2434, 0.6023928),
 (1409, 0.5973759),
 (1520, 0.5920347),
 (1861, 0.5845984),
 (2056, 0.5423897),
 (47, 0.5418963),
 (935, 0.5396448),
 (574, 0.5377094),
 (942, 0.5068472),
 (1164, 0.48870423)]

In [27]:
#title_to_idx를 뒤집어, index로부터 title 이름을 얻는 dict를 생성한다.
idx_to_title = {v:k for k, v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['lion king, the (1994)',
 'beauty and the beast (1991)',
 'aladdin (1992)',
 'mulan (1998)',
 'hunchback of notre dame, the (1996)',
 'tarzan (1999)',
 'hercules (1997)',
 'anastasia (1997)',
 'little mermaid, the (1989)',
 'antz (1998)',
 'pocahontas (1995)',
 'cinderella (1950)',
 'snow white and the seven dwarfs (1937)',
 'dumbo (1941)',
 'fantasia (1940)']

In [28]:
#위의 코드를 함수로 만들고 확인
def get_similar_title(title_name:str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

print('슝')

슝


# 9) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [29]:
user = user_to_idx['Han']
# recommend에서는 user*item CSR Matrix를 받는다.
title_recommended = als_model.recommend(user, csr_data, N = 20, filter_already_liked_items = True)
title_recommended

[(2845, 0.4882979),
 (568, 0.43409067),
 (575, 0.38566345),
 (33, 0.3687278),
 (2114, 0.30493698),
 (1689, 0.24604957),
 (2081, 0.21656285),
 (1409, 0.20951313),
 (2056, 0.19224808),
 (2506, 0.18826544),
 (342, 0.18777956),
 (1861, 0.18636318),
 (532, 0.18474928),
 (1147, 0.18325582),
 (1242, 0.17557156),
 (2434, 0.17063268),
 (304, 0.16900653),
 (725, 0.16735642),
 (1342, 0.16473085),
 (482, 0.16284075)]

In [30]:
[idx_to_title[i[0]] for i in title_recommended]

['toy story 2 (1999)',
 'aladdin (1992)',
 'beauty and the beast (1991)',
 'babe (1995)',
 "bug's life, a (1998)",
 'mulan (1998)',
 'pleasantville (1998)',
 'hercules (1997)',
 'antz (1998)',
 'iron giant, the (1999)',
 'forrest gump (1994)',
 'little mermaid, the (1989)',
 'nightmare before christmas, the (1993)',
 'groundhog day (1993)',
 '101 dalmatians (1996)',
 'tarzan (1999)',
 'santa clause, the (1994)',
 'hunchback of notre dame, the (1996)',
 'liar liar (1997)',
 'mrs. doubtfire (1993)']

# 회고록

* 좋아하는 영화 목록으로 다 2000년대 이후의 영화를 넣었더니 내 이름 벡터(user_id=Han)와 겨울왕국의 내적이 0.00숫자 대가 나와서 유의미한 내적 결과가 나오지 않았고, 새로운 다른 영화 하나와의 내적 결과 또한 비슷한 수준으로 좋지 않았다.
* 그래서 내가 좋아하는 영화 목록에 1990년대의 영화들을 입력했고 내적했는데 또한 결과가 좋지 않았다. 그 이유인 즉슨, 입력했던 the lion king (1994)영화에서 the 부분이 뒤로 빠져 있는 채로 데이터 셋에 존재했기 때문이다.
* 목록으로 뽑혀 나온 영화 데이터셋을 보니 'the mask'일 경우 'mask, the' 이런 식으로 title이 지정되어 있는 것을 발견했고, 라이언킹 또한 타이틀을 그렇게 바꾸었다.
* 그 이후 Han(user_id)벡터와 라이언킹을 내적하니 0.45853296의 결과가 나왔고,  Han(user_id)벡터와 beauty and the beast(1991)을 내적하니0.38566348의 유의미한 결과가 나왔다.